In [ ]:
#Creating a virtual enviroment with Python 3.7 and TensorFlow1.15
!conda create -n py37TF python=3.7 anaconda
!conda activate py37TF
!conda install tensorflow=1.15 -c anaconda
#!pip install tensorflow=1.15
!conda install tensorflow-hub  -c anaconda
#!pip install tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd

#Loading the conspiracy dataset.
path="./Consp-vs-Critical.csv"
df = pd.read_csv(path, sep="\t")

#Loading the pretrained ELMo model
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

# Extract ELMo features
embeddings = elmo(df["text"].tolist(), signature="default", as_dict=True)
print(embeddings['elmo'])

#Determinig the max length sequence
max_l= max([len(y.split()) for y in df["text"].tolist()])
print("Max sequence's length:", max_l)


with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    #Getting the represetation for each word in the sequence
    elmo_res = sess.run(embeddings["elmo"])
    print(elmo_res.shape)
    #Getting the repreentation of the entire sequence
    default_res = sess.run(embeddings["default"])
    print(default_res.shape)


In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')
#The max_length parameter depends on the  texts' length in the dataset
#Examples of the tokenization performed by BERT·
example_text = ["Proof has emerged showing that death from Wuhan coronavirus 'vaccine' is not always an immediate thing .",
                "The injections are loaded with Adjuvant i.e. poisons like mercury , aluminum , and bacteria ."]
tokenized_text = tokenizer.tokenize(example_text[0])
print(tokenized_text)
tokenized_text = tokenizer.tokenize(example_text[1])
print(tokenized_text)
bert_input = tokenizer(example_text, padding='max_length', max_length = 30,truncation=True, return_tensors="pt")
# Transform tokens to vocabulary indexes
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#BERT WORKS WITH THESE REPRESENTATIONS OBTAINED FOR THE TOKENS
print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])
#To transform the sequence of token indices to words in the texts
example_text = tokenizer.decode(bert_input.input_ids[0])
print(example_text)
example_text = tokenizer.decode(bert_input.input_ids[1])
print(example_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

['proof', 'has', 'emerged', 'showing', 'that', 'death', 'from', 'wu', '##han', 'corona', '##virus', "'", 'vaccine', "'", 'is', 'not', 'always', 'an', 'immediate', 'thing', '.']
['the', 'injection', '##s', 'are', 'loaded', 'with', 'ad', '##ju', '##vant', 'i', '.', 'e', '.', 'poison', '##s', 'like', 'mercury', ',', 'aluminum', ',', 'and', 'bacteria', '.']
tensor([[  101,  6947,  2038,  6003,  4760,  2008,  2331,  2013,  8814,  4819,
         21887, 23350,  1005, 17404,  1005,  2003,  2025,  2467,  2019,  6234,
          2518,  1012,   102,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1996, 13341,  2015,  2024,  8209,  2007,  4748,  9103, 18941,
          1045,  1012,  1041,  1012,  9947,  2015,  2066,  8714,  1010, 13061,
          1010,  1998, 10327,  1012,   102,     0,     0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
import pandas as pd
path="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/training/dataset_oppositional/Consp-vs-Critical.csv"
df = pd.read_csv(path, sep="\t")
#The max_length parameter depends on the  texts' length in the dataset
#Path of the trainig datataset
X_texts =  df["text"]
example_text= X_texts.to_list()
bert_input = tokenizer(example_text, padding='max_length', max_length = 100,
                      truncation=True, return_tensors="pt")
model = BertModel.from_pretrained('bert-base-uncased')
#Setting the evaluation mode, this option does not make gradient updating
model.eval()
# Send the data to GPU
bert_input = bert_input.to('cuda')
model.to('cuda')
with torch.no_grad():
   outputs = model(**bert_input)
   # Transformers models always return tuples.
   # Here, the first element corresponds to the vectors in the output of the last BERT layer.
   encoded_layers = outputs[0]
   print(encoded_layers.size())
   #Here we obtain the embedding of the CLS tokens for each input text.
   #This representation serves as a contextual embedding of the texts.
   cls_vector = encoded_layers[:,0,:]
   print(cls_vector.size())
   i=1
   for vect in cls_vector.cpu().detach().numpy():
    print(f"This is the CLS vector for the document D{i}", vect )
    i+=1
   #Vector associated with the CLS token of the first text in the entry.
   cls_vector = cls_vector.cpu().detach().numpy()[0]
   print(len(cls_vector))

Streaming output truncated to the last 5000 lines.
  4.13810581e-01  2.30804220e-01 -1.87874869e-01  3.44812721e-01
  2.81062037e-01  4.47978973e-01 -4.37920541e-01 -3.18655461e-01
 -5.86422496e-02  1.31898344e-01  2.21938089e-01  3.08126926e-01
 -6.58656645e+00 -8.88100341e-02 -1.57116726e-01  4.10243720e-01
  6.87559601e-03 -3.06918949e-01  1.53994858e-01  1.48623571e-01
 -1.11997508e-01 -1.93526479e-03  3.90439987e-01 -1.56040564e-01
 -3.47521394e-01 -2.37715513e-01  6.60021067e-01  7.06938088e-01]
This is the CLS vector for the document D75 [ 9.62395146e-02 -2.36992493e-01 -2.79342923e-02 -2.77807295e-01
 -2.06740871e-01 -3.96647781e-01  2.83933699e-01  4.75741655e-01
  8.51679742e-02 -5.81899226e-01 -1.43851727e-01 -5.32669909e-02
 -6.14267401e-02  2.96912372e-01  5.63004255e-01  1.48917899e-01
 -2.92193234e-01  5.92529178e-01 -5.60717806e-02 -5.98296039e-02
  2.88924035e-02 -7.97772169e-01  2.53517777e-01 -1.40880018e-01
  9.97124016e-02 -2.65348822e-01  6.11480623e-02 -5.0838506

In [ ]:
import torch
from transformers import DebertaTokenizer
tokenizer= DebertaTokenizer.from_pretrained('microsoft/deberta-base')
#The max_length parameter depends on the  texts' length in the dataset
#Examples of the tokenization performed by DeBERTa
example_text = ["Proof has emerged showing that death from Wuhan coronavirus 'vaccine' is not always an immediate thing .",
                "The injections are loaded with Adjuvant i.e. poisons like mercury , aluminum , and bacteria ."]
tokenized_text = tokenizer.tokenize(example_text[0])
print(tokenized_text)
tokenized_text = tokenizer.tokenize(example_text[1])
print(tokenized_text)
deberta_input = tokenizer(example_text, padding='max_length', max_length = 30,
                      truncation=True, return_tensors="pt")

# Transform tokens to vocabulary indexes
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#BERT WORKS WITH THESE REPRESENTATIONS OBTAINED FOR THE TOKENS
print(deberta_input['input_ids'])


print(deberta_input['attention_mask'])
#To transform the sequence of token indices to words in the texts
example_text = tokenizer.decode(deberta_input.input_ids[0])
print(example_text)
example_text = tokenizer.decode(deberta_input.input_ids[1])
print(example_text)


['Proof', 'Ġhas', 'Ġemerged', 'Ġshowing', 'Ġthat', 'Ġdeath', 'Ġfrom', 'ĠW', 'uh', 'an', 'Ġcoron', 'av', 'irus', "Ġ'", 'vacc', 'ine', "'", 'Ġis', 'Ġnot', 'Ġalways', 'Ġan', 'Ġimmediate', 'Ġthing', 'Ġ.']
['The', 'Ġinjections', 'Ġare', 'Ġloaded', 'Ġwith', 'ĠAd', 'ju', 'vant', 'Ġi', '.', 'e', '.', 'Ġpoisons', 'Ġlike', 'Ġmercury', 'Ġ,', 'Ġaluminum', 'Ġ,', 'Ġand', 'Ġbacteria', 'Ġ.']
tensor([[    1, 46484,    34,  4373,  2018,    14,   744,    31,   305,  2957,
           260, 34377,  1469, 19473,   128, 39150,   833,   108,    16,    45,
           460,    41,  3169,   631,   479,     2,     0,     0,     0,     0],
        [    1,   133, 25747,    32,  7973,    19,  1614,  7488, 21454,   939,
             4,   242,     4, 45265,   101, 25755,  2156,  9757,  2156,     8,
          9436,   479,     2,     0,     0,     0,     0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
import torch
from transformers import DebertaTokenizer, DebertaModel
tokenizer=DebertaTokenizer.from_pretrained('microsoft/deberta-base')
import pandas as pd
path="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/training/dataset_oppositional/Consp-vs-Critical.csv"
df = pd.read_csv(path, sep="\t")
#The max_length parameter depends on the  texts' length in the dataset
#Path of the trainig datataset
X_texts =  df["text"]
example_text= X_texts.to_list()
deberta_input = tokenizer(example_text, padding='max_length', max_length = 50,
                      truncation=True, return_tensors="pt")
model = DebertaModel.from_pretrained('microsoft/deberta-base')
#Setting the evaluation mode, this option does not make gradient updating
model.eval()
# Send the data to GPU
deberta_input = deberta_input.to('cuda')
model.to('cuda')
with torch.no_grad():
   outputs = model(**deberta_input)
   # Transformers models always return tuples.
   # Here, the first element corresponds to the vectors in the output of the last DeBERTa layer.
   encoded_layers = outputs[0]
   print(encoded_layers.size())
   #Here we obtain the embedding of the CLS tokens for each input text.
   #This representation serves as a contextual embedding of the texts.
   cls_vector = encoded_layers[:,0,:]
   print(cls_vector.size())
   #Vector associated with the CLS token of the first text in the entry.
   cls_vector = cls_vector.cpu().detach().numpy()[0]
   print(len(cls_vector))

tensor([[    1,   133,  7597,  ...,     0,     0,     0],
        [    1, 42050,  4832,  ...,     0,     0,     0],
        [    1, 19663, 17767,  ...,     0,     0,     0],
        ...,
        [    1,   113,   221,  ...,     0,     0,     0],
        [    1, 14009,  6969,  ...,     0,     0,     0],
        [    1,   970,    44,  ...,     0,     0,     0]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

torch.Size([100, 50, 768])
torch.Size([100, 768])
768


In [ ]:
import torch
from transformers import RobertaTokenizer
tokenizer= RobertaTokenizer.from_pretrained('roberta-base')
#The max_length parameter depends on the  texts' length in the dataset
#Examples of the tokenization performed by RoBERTa
example_text = ["Proof has emerged showing that death from Wuhan coronavirus 'vaccine' is not always an immediate thing .",
                "The injections are loaded with Adjuvant i.e. poisons like mercury , aluminum , and bacteria ."]
tokenized_text = tokenizer.tokenize(example_text[0])
print(tokenized_text)
tokenized_text = tokenizer.tokenize(example_text[1])
print(tokenized_text)
roberta_input = tokenizer(example_text, padding='max_length', max_length = 30,
                      truncation=True, return_tensors="pt")

# Transform tokens to vocabulary indexes
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#RoBERTa WORKS WITH THESE REPRESENTATIONS OBTAINED FOR THE TOKENS
print(roberta_input['input_ids'])
print(roberta_input['attention_mask'])
#To transform the sequence of token indices to words in the texts
example_text = tokenizer.decode(roberta_input.input_ids[0])
print(example_text)
example_text = tokenizer.decode(roberta_input.input_ids[1])
print(example_text)

['Proof', 'Ġhas', 'Ġemerged', 'Ġshowing', 'Ġthat', 'Ġdeath', 'Ġfrom', 'ĠW', 'uh', 'an', 'Ġcoron', 'av', 'irus', "Ġ'", 'vacc', 'ine', "'", 'Ġis', 'Ġnot', 'Ġalways', 'Ġan', 'Ġimmediate', 'Ġthing', 'Ġ.']
['The', 'Ġinjections', 'Ġare', 'Ġloaded', 'Ġwith', 'ĠAd', 'ju', 'vant', 'Ġi', '.', 'e', '.', 'Ġpoisons', 'Ġlike', 'Ġmercury', 'Ġ,', 'Ġaluminum', 'Ġ,', 'Ġand', 'Ġbacteria', 'Ġ.']
tensor([[    0, 46484,    34,  4373,  2018,    14,   744,    31,   305,  2957,
           260, 34377,  1469, 19473,   128, 39150,   833,   108,    16,    45,
           460,    41,  3169,   631,   479,     2,     1,     1,     1,     1],
        [    0,   133, 25747,    32,  7973,    19,  1614,  7488, 21454,   939,
             4,   242,     4, 45265,   101, 25755,  2156,  9757,  2156,     8,
          9436,   479,     2,     1,     1,     1,     1,     1,     1,     1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaModel
tokenizer=RobertaTokenizer.from_pretrained('roberta-base')
import pandas as pd
path="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/training/dataset_oppositional/Consp-vs-Critical.csv"
df = pd.read_csv(path, sep="\t")
#The max_length parameter depends on the  texts' length in the dataset
#Path of the trainig datataset
X_texts =  df["text"]
example_text= X_texts.to_list()
roberta_input = tokenizer(example_text, padding='max_length', max_length = 50,
                      truncation=True, return_tensors="pt")
model = RobertaModel.from_pretrained('roberta-base')
#Setting the evaluation mode, this option does not make gradient updating
model.eval()
# Send the data to GPU
roberta_input = roberta_input.to('cuda')
model.to('cuda')
with torch.no_grad():
   outputs = model(**roberta_input)
   # Transformers models always return tuples.
   # Here, the first element corresponds to the vectors in the output of the last RoBERTa layer.
   encoded_layers = outputs[0]
   print(encoded_layers.size())
   #Here we obtain the embedding of the CLS tokens for each input text.
   #This representation serves as a contextual embedding of the texts.
   cls_vector = encoded_layers[:,0,:]
   print(cls_vector.size())
   #Vector associated with the CLS token of the first text in the entry.
   cls_vector = cls_vector.cpu().detach().numpy()[0]
   print(len(cls_vector))

tensor([[    0,   133,  7597,  ...,     1,     1,     1],
        [    0, 42050,  4832,  ...,     1,     1,     1],
        [    0, 19663, 17767,  ...,     1,     1,     1],
        ...,
        [    0,   113,   221,  ...,     1,     1,     1],
        [    0, 14009,  6969,  ...,     1,     1,     1],
        [    0,   970,    44,  ...,     1,     1,     1]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([100, 50, 768])
torch.Size([100, 768])
768


In [ ]:
import torch
from transformers import BertTokenizer
tokenizer= BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
#The max_length parameter depends on the  texts' length in the dataset
#Examples of the tokenization performed by BETO·
example_text = ['Usando BETO en clases de ciencia de datos de la universidad politécnica.', "Los estudiantes de este grado son muy aplicados y estudiosos."]
tokenized_text = tokenizer.tokenize(example_text[0])
print(tokenized_text)
tokenized_text = tokenizer.tokenize(example_text[1])
print(tokenized_text)
beto_input = tokenizer(example_text,padding='max_length', max_length = 20,
                      truncation=False, return_tensors="pt")


# Transform tokens to vocabulary indexes
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#BETO WORKS WITH THESE REPRESENTATIONS OBTAINED FOR THE TOKENS
print(beto_input['input_ids'])
print(beto_input['token_type_ids'])
print(beto_input['attention_mask'])
#To transform the sequence of token indices to words in the texts
example_text = tokenizer.decode(beto_input.input_ids[0])
print(example_text)
example_text = tokenizer.decode(beto_input.input_ids[1])
print(example_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

['usando', 'bet', '##o', 'en', 'clases', 'de', 'ciencia', 'de', 'datos', 'de', 'la', 'universidad', 'politécn', '##ica', '.']
['los', 'estudiantes', 'de', 'este', 'grado', 'son', 'muy', 'aplicados', 'y', 'estudiosos', '.']
tensor([[    4,  6275, 20768, 30957,  1035,  5788,  1009,  4998,  1009,  2783,
          1009,  1032,  2862, 28363,  1172,  1008,     5,     1,     1,     1],
        [    4,  1067,  4932,  1009,  1277,  5155,  1318,  1355, 25697,  1040,
         27322,  1008,     5,     1,     1,     1,     1,     1,     1,     1]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])
[CLS] usando beto en clases de ciencia de datos de la universidad politécnica. [SEP] [PAD] [PAD] [PAD]
[CLS] los estudiantes de este grado son muy aplicados y estudiosos. [SEP]

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
modelname='dccuchile/bert-base-spanish-wwm-uncased'
tokenizer=BertTokenizer.from_pretrained(modelname)
import pandas as pd
path="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/training/dataset_oppositional/Consp-vs-Critical.csv"
df = pd.read_csv(path, sep="\t")
#The max_length parameter depends on the  texts' length in the dataset
#Path of the training dataset
X_texts =  df["text"]
example_text= X_texts.to_list()
beto_input = tokenizer(example_text, padding='max_length', max_length = 50, truncation=True, return_tensors="pt")

model = BertModel.from_pretrained(modelname)
#Setting the evaluation mode, this option does not make gradient updating
model.eval()
# Send the data to GPU
beto_input = beto_input.to('cuda')
model.to('cuda')
with torch.no_grad():
  outputs = model(**beto_input)
  # Transformers models always return tuples.
  # Here, the first element corresponds to the vectors in the output of the last BETO layer.
  encoded_layers = outputs[0]
  print(encoded_layers.size())
  #Here we obtain the embedding of the CLS tokens for each input text.
  #This representation serves as a contextual embedding of the texts.
  cls_vector = encoded_layers[:,0,:]
  print(cls_vector.size())
  #Vector associated with the CLS token of the first text in the entry.
  cls_vector = cls_vector.cpu().detach().numpy()[0]
  print(len(cls_vector))

tensor([[    4,  2258, 19020,  ...,     1,     1,     1],
        [    4,  1005, 16859,  ...,     1,     1,     1],
        [    4,  9016,  1956,  ...,     1,     1,     1],
        ...,
        [    4,  1100,  1584,  ...,  1100,  1100,     5],
        [    4,  4927, 21618,  ...,     1,     1,     1],
        [    4, 20525,     3,  ...,  1008,     5,     1]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0]])


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([100, 50, 768])
torch.Size([100, 768])
768


In [ ]:
import torch
from transformers import RobertaTokenizer
modelname='PlanTL-GOB-ES/roberta-base-bne'
tokenizer= RobertaTokenizer.from_pretrained(modelname)
#The max_length parameter depends on the  texts' length in the dataset
#Examples of the tokenization performed by BETO·
example_text = ['Usando BETO en clases de ciencia de datos de la universidad politécnica.', "Los estudiantes de este grado son muy aplicados y estudiosos."]
tokenized_text = tokenizer.tokenize(example_text[0])
print(tokenized_text)
tokenized_text = tokenizer.tokenize(example_text[1])
print(tokenized_text)
roberta_sp_input = tokenizer(example_text,padding='max_length', max_length = 20, truncation=False, return_tensors="pt")
# Transform tokens to vocabulary indexes
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#RoBERTa WORKS WITH THESE REPRESENTATIONS OBTAINED FOR THE TOKENS
print(roberta_sp_input['input_ids'])
print(roberta_sp_input['attention_mask'])
#To transform the sequence of token indices to words in the texts
example_text = tokenizer.decode(roberta_sp_input.input_ids[0])
print(example_text)
example_text = tokenizer.decode(roberta_sp_input.input_ids[1])
print(example_text)

['Us', 'ando', 'ĠBE', 'TO', 'Ġen', 'Ġclases', 'Ġde', 'Ġciencia', 'Ġde', 'Ġdatos', 'Ġde', 'Ġla', 'Ġuniversidad', 'Ġpolit', 'Ã©cnica', '.']
['Los', 'Ġestudiantes', 'Ġde', 'Ġeste', 'Ġgrado', 'Ġson', 'Ġmuy', 'Ġaplicados', 'Ġy', 'Ġestudiosos', '.']
tensor([[    0,  8940,   485, 45708,  2201,   334,  3885,   313,  5456,   313,
          1319,   313,   332,  7057, 35612, 15090,    68,     2,     1,     1],
        [    0,   713,  4550,   313,   533,  5502,   644,   727, 34006,   342,
         39024,    68,     2,     1,     1,     1,     1,     1,     1,     1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])
<s>Usando BETO en clases de ciencia de datos de la universidad politécnica.</s><pad><pad>
<s>Los estudiantes de este grado son muy aplicados y estudiosos.</s><pad><pad><pad><pad><pad><pad><pad>


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaModel
modelname='PlanTL-GOB-ES/roberta-base-bne'
tokenizer=RobertaTokenizer.from_pretrained(modelname)
import pandas as pd
path="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/training/dataset_oppositional/Consp-vs-Critical.csv"
df = pd.read_csv(path, sep="\t")
#The max_length parameter depends on the  texts' length in the dataset
#Path of the training dataset
X_texts =  df["text"]
example_text= X_texts.to_list()
roberta_sp_input = tokenizer(example_text, padding='max_length', max_length = 50, truncation=True, return_tensors="pt")

model = RobertaModel.from_pretrained(modelname)
#Setting the evaluation mode, this option does not make gradient updating
model.eval()
# Send the data to GPU
roberta_sp_input = roberta_sp_input.to('cuda')
model.to('cuda')
with torch.no_grad():
  outputs = model(**roberta_sp_input)
  # Transformers models always return tuples.
  # Here, the first element corresponds to the vectors in the output of the last RoBERTa layer.
  encoded_layers = outputs[0]
  print(encoded_layers.size())
  #Here we obtain the embedding of the CLS tokens for each input text.
  #This representation serves as a contextual embedding of the texts.
  cls_vector = encoded_layers[:,0,:]
  print(cls_vector.size())
  #Vector associated with the CLS token of the first text in the entry.
  cls_vector = cls_vector.cpu().detach().numpy()[0]
  print(len(cls_vector))

tensor([[    0,  6649,  9399,  ...,     1,     1,     1],
        [    0,   109,  4284,  ...,     1,     1,     1],
        [    0,    96, 26800,  ...,     1,     1,     1],
        ...,
        [    0,    56,  2970,  ...,   726,  8844,     2],
        [    0, 34724, 44237,  ...,  5219,   143,     2],
        [    0,   106, 35192,  ...,  2874,  1247,     2]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([100, 50, 768])
torch.Size([100, 768])
768
